In [2]:
import time
import pandas as pd
import ibm_db
import logging
from sqlalchemy import create_engine
import os
import glob
import json
from datetime import datetime
import datetime as dt 

start_time = time.time()

conn = ibm_db.connect("DATABASE=AYB_APPL;HOSTNAME=10.143.16.244;PORT=50000;PROTOCOL=TCPIP;UID=IOT_DATA;PWD=asd23fgh;", "", "")
if conn:
    print("Connected to the database")


    
# 1) LOCATIONの最新のデータをINSERT/UPDATE


#1.  最新フォルダの全CSVのPATHをまとめる。
directory =  r"C:\Users\00220401626\Desktop\FMS\CsvData"
file_pattern = '20*'  
files = glob.glob(os.path.join(directory, file_pattern))
files.sort()
folder_path = files[-1]
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))



#2. 最新のCSVファイルを探す。
if csv_files:  
    latest_csv_file = max(csv_files, key=os.path.getmtime)  
    df = pd.read_csv(latest_csv_file, encoding='cp932',dtype= object)
 
#3. CSVファイルのLocationカラムから最新の値を変数に入れる。
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    a = df['Location'].unique()
    latest_data = pd.DataFrame()
    df_concat = []
    for location in a:
        latest_location = df[df['Location'] == location].nlargest(1, 'Timestamp')
        df_concat.append(latest_location)
    latest_data = pd.concat(df_concat, ignore_index=True)
    print("Latest CSV file:", latest_csv_file)
    
else:
    print("No CSV files found in the folder.")


    
    
# 4. データがあればUPDATE, データがなければINSERTする。
merge_query = """
MERGE INTO LIQUID_PARTICLE_LAST AS target
USING (SELECT ? AS LOCATION, ? AS TS_DT, ? AS VALUE_1_0, ? AS VALUE_3_0, ? AS VALUE_5_0, ? AS VALUE_10_0, ? AS VALUE_15_0, ? AS VALUE_20_0, ? AS VALUE_25_0, ? AS VALUE_50_0 FROM SYSIBM.SYSDUMMY1) AS source
ON target.LOCATION = source.LOCATION
WHEN MATCHED THEN
    UPDATE SET target.TS_DT = source.TS_DT, 
               target.VALUE_1_0 = source.VALUE_1_0, 
               target.VALUE_3_0 = source.VALUE_3_0, 
               target.VALUE_5_0 = source.VALUE_5_0, 
               target.VALUE_10_0 = source.VALUE_10_0, 
               target.VALUE_15_0 = source.VALUE_15_0, 
               target.VALUE_20_0 = source.VALUE_20_0, 
               target.VALUE_25_0 = source.VALUE_25_0, 
               target.VALUE_50_0 = source.VALUE_50_0
WHEN NOT MATCHED THEN
    INSERT (LOCATION, TS_DT, VALUE_1_0, VALUE_3_0, VALUE_5_0, VALUE_10_0, VALUE_15_0, VALUE_20_0, VALUE_25_0, VALUE_50_0)
    VALUES (source.LOCATION, source.TS_DT, source.VALUE_1_0, source.VALUE_3_0, source.VALUE_5_0, source.VALUE_10_0, source.VALUE_15_0, source.VALUE_20_0, source.VALUE_25_0, source.VALUE_50_0);
"""

stmt = ibm_db.prepare(conn, merge_query)

for index, row in latest_data.iterrows():
    location = row['Location']
    timestamp = row['Timestamp']
    c1 = row['1.0μｍ']
    c3 = row['3.0μｍ']
    c5 = row['5.0μｍ']
    c10 = row['10.0μｍ']
    c15 = row['15.0μｍ']
    c20 = row['20.0μｍ']
    c25 = row['25.0μｍ']
    c50 = row['50.0μｍ']
    
    if ibm_db.execute(stmt, (location, timestamp, c1, c3, c5, c10, c15, c20, c25, c50)):
        print(f"Row {index + 1} merged successfully")
    else:
        print(f"Error merging row {index + 1}: {ibm_db.stmt_errormsg()}")


        
#############################################################################################################
        
# 2) JSON記録に記載された日より、以降のデータをCSVファイルごとにINSERTする

        
#　1.JSONの記録からデータベースに最後INSERTされた時刻を取得する

with open('insertion_time.json', 'r') as json_file:
    data = json.load(json_file)
    inserted_time = dt.datetime.strptime(data['insertion_time'], '%Y-%m-%d %H:%M:%S')
    inserted_day = inserted_time.strftime('%Y%m%d')
    time_to_compare = data['insertion_time']

    

#  2.最新のフォルダから,全CSVのPATHをまとめる

directory =  r"C:\Users\00220401626\Desktop\FMS\CsvData"
file_pattern = '20*'  
files = glob.glob(os.path.join(directory, file_pattern))
files.sort()
folder_path = files[-1]
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))




# 　3.JSON記録に記載された日と、それより以降のCSV1をまとめる  
selected_csv_files=[]
selected_csv_files = [file for file in csv_files if int(file[-12:-4]) >= int(inserted_day)]




#   4.JSON記録に記載された日より以降のCSV1を１つずつINSERTする  
if selected_csv_files:
    
    for i in selected_csv_files:
        df = pd.read_csv(fr"{i}", encoding='cp932', dtype= object)
        df['Timestamp'] = df['Timestamp'].str.replace('/', '-', regex=False)
        
        a = df[df['Timestamp'] >  time_to_compare]
        data = tuple(tuple(row) for row in a.values )
           
        b = ""
        for row in data:
            b += str(row) +","
        b = b[:-1]
        
        insert_query = f"INSERT INTO LIQUID_PARTICLE_TEST (Location, TS_DT, VALUE_1_0, VALUE_3_0, VALUE_5_0, VALUE_10_0, VALUE_15_0, VALUE_20_0, VALUE_25_0, VALUE_50_0) VALUES {b}"
        stmt = ibm_db.prepare(conn, insert_query)
        
        try:
            if ibm_db.execute(stmt, b):
                print(f"{i} : Inserted successfully")     
        except:
            print(f"({i}) is already updated")
                
        
#   5.時間を JSON ファイルに保存する   
    
        insertion_time = a['Timestamp'].max()
        json_data = {'insertion_time': insertion_time}
        with open('insertion_time.json', 'w') as json_file:
            json.dump(json_data, json_file, indent=4)
    print('All CSV files are inserted')
elif not selected_csv_files:
    print("Database is already updated")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time spent: {elapsed_time} seconds")

Connected to the database
Latest CSV file: C:\Users\00220401626\Desktop\FMS\CsvData\2026\20231022.csv
Row 1 merged successfully
Row 2 merged successfully
Row 3 merged successfully
Row 4 merged successfully
Row 5 merged successfully
Row 6 merged successfully
Row 7 merged successfully
Row 8 merged successfully
Row 9 merged successfully
Row 10 merged successfully
Row 11 merged successfully
Row 12 merged successfully
Row 13 merged successfully
C:\Users\00220401626\Desktop\FMS\CsvData\2026\20231001.csv : Inserted successfully
C:\Users\00220401626\Desktop\FMS\CsvData\2026\20231002.csv : Inserted successfully
C:\Users\00220401626\Desktop\FMS\CsvData\2026\20231003.csv : Inserted successfully
C:\Users\00220401626\Desktop\FMS\CsvData\2026\20231004.csv : Inserted successfully
C:\Users\00220401626\Desktop\FMS\CsvData\2026\20231005.csv : Inserted successfully
C:\Users\00220401626\Desktop\FMS\CsvData\2026\20231006.csv : Inserted successfully
C:\Users\00220401626\Desktop\FMS\CsvData\2026\20231007.csv

In [ ]:
# if os.path.exists(json_file_path):
#     with open(json_file_path, 'r') as json_file:
#         data = json.load(json_file)
#         inserted_time = dt.datetime.strptime(data['insertion_time'], '%Y-%m-%d %H:%M:%S')
#         inserted_day = inserted_time.strftime('%Y%m%d')
#         time_to_compare = data['insertion_time']
#         print(f"Last Insertion time: {time_to_compare}")
#     directory =  r"C:\Users\00220401626\Desktop\FMS\CsvData"
#     file_pattern = '20*'  
#     files = glob.glob(os.path.join(directory, file_pattern))
#     files.sort()
#     folder_path = files[-1]
#     csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
#     selected_csv_files=[]
#     selected_csv_files = [file for file in csv_files if int(file[-12:-4]) >= int(inserted_day)]
#     selected_csv_files
# else:
#     print("JSON file does not exist. All CSV files below are going to be inserted into the database.")
#     logging.info("JSON file does not exist. All CSV files below are going to be inserted into the database.")
#     directory =  r"C:\Users\00220401626\Desktop\FMS\CsvData"
#     file_pattern = '20*'  
#     files = glob.glob(os.path.join(directory, file_pattern))
#     files.sort()
#     folder_path = files[-1]
#     selected_csv_files = glob.glob(os.path.join(folder_path, '*.csv'))


In [ ]:
import time
import pandas as pd
import ibm_db
import logging
import os
import glob
import json
from datetime import datetime
import datetime as dt 
import shutil


json_path = r'C:\Users\00220401626\Desktop\test\config.json'
with open(json_path) as json_file:
    config = json.load(json_file)

database = config['database']
host = config['host']
port = config['port']
protocol = config['protocol']
uid = config['uid']
pwd = config['pwd']
pc_csv = config['pc_csv']
local_csv = config['local_csv']
log = config['log']
insertion_time = config['insertion_time']

time = str(insertion_time)
inserted_time = dt.datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
inserted_day = inserted_time.strftime('%Y%m%d')

print(inserted_time)
print(inserted_day)

logging.basicConfig(filename=log, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def copy_local():
    if insertion_time:    
        csv_files = glob.glob(os.path.join(pc_csv,'*.csv'))
        selected_csv_paths = [file for file in csv_files if int(file[-12:-4]) >= int(inserted_day)]
    else:
        selected_csv_paths = glob.glob(os.path.join(pc_csv,'*.csv'))
        
    for source_path in selected_csv_paths:
        if os.path.exists(source_path):  
            file_name = os.path.basename(source_path)
            destination_path = os.path.join(local_csv, file_name)
            shutil.copy(source_path, destination_path)
            print(f"'{source_path}' copied.")
        else:
            print(f"Source file '{source_path}' does not exist.")
    
def connect_db():
    dsn = f"DATABASE={database};HOSTNAME={host};PORT={port};PROTOCOL={protocol};UID={uid};PWD={pwd}"
    conn = ibm_db.connect(dsn, "", "")
    if conn:
        logging.info("Connected to the database")
        print("Connected to the database")
    else:
        logging.info("Failed to connect to the database") 
        print("Not connected to the database")
    return conn

def latest_data(local_csv):
    try:
        csv_files = glob.glob(os.path.join(local_csv, '*.csv'))
        if csv_files:
            latest_csv_file = max(csv_files, key=os.path.getmtime)                                                                #max(int(csv_files[-12:-4]))
            df = pd.read_csv(f"{latest_csv_file}", encoding='cp932', dtype=object)
        
        if not df.empty:
            df['Timestamp'] = pd.to_datetime(df['Timestamp'])
            unique = df['Location'].unique()
            
            df_concat = []
            for location in unique:
                latest_location = df[df['Location'] == location].nlargest(1, 'Timestamp')
                df_concat.append(latest_location)
                
            latest_data = pd.concat(df_concat, ignore_index=True)
            print(latest_data)
            merge_query = """
                MERGE INTO LIQUID_PARTICLE_LAST AS target
                USING (SELECT ? AS LOCATION, ? AS TS_DT, ? AS VALUE_1_0, ? AS VALUE_3_0, ? AS VALUE_5_0, ? AS VALUE_10_0, ? AS VALUE_15_0, ? AS VALUE_20_0, ? AS VALUE_25_0, ? AS VALUE_50_0 FROM SYSIBM.SYSDUMMY1) AS source
                ON target.LOCATION = source.LOCATION
                WHEN MATCHED THEN
                    UPDATE SET target.TS_DT = source.TS_DT, 
                               target.VALUE_1_0 = source.VALUE_1_0, 
                               target.VALUE_3_0 = source.VALUE_3_0, 
                               target.VALUE_5_0 = source.VALUE_5_0, 
                               target.VALUE_10_0 = source.VALUE_10_0, 
                               target.VALUE_15_0 = source.VALUE_15_0, 
                               target.VALUE_20_0 = source.VALUE_20_0, 
                               target.VALUE_25_0 = source.VALUE_25_0, 
                               target.VALUE_50_0 = source.VALUE_50_0
                WHEN NOT MATCHED THEN
                    INSERT (LOCATION, TS_DT, VALUE_1_0, VALUE_3_0, VALUE_5_0, VALUE_10_0, VALUE_15_0, VALUE_20_0, VALUE_25_0, VALUE_50_0)
                    VALUES (source.LOCATION, source.TS_DT, source.VALUE_1_0, source.VALUE_3_0, source.VALUE_5_0, source.VALUE_10_0, source.VALUE_15_0, source.VALUE_20_0, source.VALUE_25_0, source.VALUE_50_0);
                """
        
            stmt = ibm_db.prepare(conn, merge_query)
        
            for index, row in latest_data.iterrows():
                location = row['Location']
                timestamp = row['Timestamp']
                c1 = row['1.0μｍ']
                c3 = row['3.0μｍ']
                c5 = row['5.0μｍ']
                c10 = row['10.0μｍ']
                c15 = row['15.0μｍ']
                c20 = row['20.0μｍ']
                c25 = row['25.0μｍ']
                c50 = row['50.0μｍ']
        
                try:
                    if ibm_db.execute(stmt, (location, timestamp, c1, c3, c5, c10, c15, c20, c25, c50)):
                        logging.info(f"Row {index + 1} merged successfully")
                    else:
                        error_message = f"Error merging row {index + 1}: {ibm_db.stmt_errormsg()}"
                        logging.error(error_message)
                except Exception as e:
                    error_message = f"Exception merging row {index + 1}: {str(e)}"
                    logging.error(error_message)

    except Exception as e:
        print(f"An error occurred: {e}")

    
def all_data(local_csv):
    csv_files = glob.glob(os.path.join(local_csv, '*.csv'))
    if csv_files:  
        for csv in csv_files:
            df = pd.read_csv(fr"{csv}", encoding='cp932', dtype= object)
            df['Timestamp'] = df['Timestamp'].str.replace('/', '-', regex=False)
          
            config['insertion_time'] = str(df['Timestamp'].max())
            json_data = json.dumps(config, indent=2)
            
            print(json_data)
            
            df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y-%m-%d %H:%M:%S') 
            
            if os.path.exists(json_path) and inserted_time:
                df = df[df['Timestamp'] > inserted_time]
                data = tuple(tuple(row) for row in df.values)
                values = ",".join(map(str, data))
            
                insert_query = f"INSERT INTO LIQUID_PARTICLE_TEST (Location, TS_DT, VALUE_1_0, VALUE_3_0, VALUE_5_0, VALUE_10_0, VALUE_15_0, VALUE_20_0, VALUE_25_0, VALUE_50_0) VALUES {values}"
                stmt = ibm_db.prepare(conn, insert_query)
            
                try:
                    if ibm_db.execute(stmt, values):
                        logging.info(f"{csv} : Inserted successfully")
                        print(f"{csv} : Inserted successfully")
                            
                        with open(json_path, 'w') as output_file:
                            output_file.write(json_data)
                except:
                    logging.info(f"{csv} : is already updated")
                    print(f"{csv} : is already updated")
              
            else:
                data = tuple(tuple(row) for row in df.values)
                values = ",".join(map(str, data))
            
                insert_query = f"INSERT INTO LIQUID_PARTICLE_TEST (Location, TS_DT, VALUE_1_0, VALUE_3_0, VALUE_5_0, VALUE_10_0, VALUE_15_0, VALUE_20_0, VALUE_25_0, VALUE_50_0) VALUES {values}"
                stmt = ibm_db.prepare(conn, insert_query)
            
                try:
                    if ibm_db.execute(stmt, values):
                        logging.info(f"{csv} : Inserted successfully")
                        
                        with open(json_path, 'w') as output_file:
                            output_file.write(json_data)
                except:
                    logging.info(f"{csv} : is already updated")
                    print(f"{csv} : is already updated")
         
        logging.info('All CSV files are inserted')
    
    elif not selected_csv_files:
        logging.info("Database is already updated")


def backup_and_delete_csv_files(local_csv, nas):
    csv_files = glob.glob(os.path.join(local_csv,'*.csv'))
    for source_path in csv_files:
        if os.path.exists(source_path):  
            file_name = os.path.basename(source_path)
            destination_path = os.path.join(nas, file_name)
            shutil.copy(source_path, destination_path)
            os.remove(source_path)
            print(f"File '{file_name}' copied to NAS and deleted from local_csv.")
        else:
            print(f"Source file '{source_path}' does not exist.")

# start_time = time.time()   

copy_local()
conn = connect_db()
# latest_data(local_csv)
all_data(local_csv)
# backup_and_delete_csv_files(local_csv, nas)

# end_time = time.time()
# elapsed_time = end_time - start_time
# minutes = int(elapsed_time // 60)
# seconds = int(elapsed_time % 60)
# logging.info(f"Time spent: {minutes} minutes and {seconds} seconds")